In [2]:
import sys

class Simplex:
    def __init__(self, tokens):
        self.val = float(tokens.pop(0))
        self.dim = int(tokens.pop(0))
        self.vert = set(int(tokens.pop(0)) for _ in range(self.dim + 1))

    def __str__(self):
        return f"{{val={self.val}; dim={self.dim}; {sorted(self.vert)}}}"

In [3]:
# Return a sorted by time filtration from a file 
def read_filtration(filename):
    filtration = []
    with open(filename, "r") as f:
        tokens = []
        for line in f:
            tokens.extend(line.strip().split())
        # On lit les simplexes jusqu’à ce qu’il n’y ait plus de tokens
        while tokens:
            filtration.append(Simplex(tokens))

        filtration.sort(key=lambda s: s.val)
    return filtration

filtration = read_filtration("filtration.txt")
for simplex in filtration:
    print(simplex)

{val=1.0; dim=0; [2]}
{val=1.0; dim=0; [4]}
{val=1.0; dim=0; [1]}
{val=2.0; dim=1; [2, 4]}
{val=2.0; dim=1; [1, 2]}
{val=3.0; dim=0; [7]}
{val=4.0; dim=1; [4, 7]}
{val=4.0; dim=1; [1, 7]}
{val=5.0; dim=1; [1, 4]}
{val=6.0; dim=2; [1, 4, 7]}


In [4]:
## Question 1 - Boundary matrix
def boundary_matrix(filtration : list[Simplex]) -> list[list[int]]:

    # Sort the filtration by time insertion
    filtration = sorted(filtration, key=lambda s: s.val)

    n = len(filtration)
    
    boundary = []

    for j, simplex in enumerate(filtration):
        boundary.append([])
        for i, face in enumerate(filtration):
            if i != j and face.dim == simplex.dim - 1 and face.vert.issubset(simplex.vert):
                boundary[j].append(i)

    return boundary

print(boundary_matrix(filtration))


[[], [], [], [0, 1], [0, 2], [], [1, 5], [2, 5], [1, 2], [6, 7, 8]]


In [5]:

## Question 2,3 - Reduction algorithm
def reduce_boundary_matrix(boundary : list[list[int]]) -> list[list[int]]:
    
    reduced_boundary = boundary.copy()
    m = len(reduced_boundary)

    low = [max(col) if col else -1 for col in reduced_boundary]

    for j in range(m):
        while low[j] != -1 and low[j] in low[:j]:

            i = low.index(low[j])

            # Perform column addition (mod 2) : XOR
            reduced_boundary[j] = list(set(reduced_boundary[j]) ^ set(reduced_boundary[i]))
            low[j] = max(reduced_boundary[j]) if reduced_boundary[j] else -1

    return reduced_boundary # à optimiser

print(reduce_boundary_matrix(boundary_matrix(filtration)))

[[], [], [], [0, 1], [0, 2], [], [1, 5], [], [], [6, 7, 8]]


In [6]:

## Question 4 - Barcode extraction
def extract_barcodes(reduced_boundary : list[list[int]], filtration : list[Simplex]) -> list[tuple[int, int, int]]:

    seen_indexes = set()
    barcodes = []

    for j in range(len(reduced_boundary)):
        if reduced_boundary[j]:
            seen_indexes.add(j)
            low_j = max(reduced_boundary[j])
            seen_indexes.add(low_j)
            barcode = (filtration[low_j].dim, low_j,j)  # (index, death index, dimension)
            barcodes.append(barcode)

    print("Seen indexes:", seen_indexes)
    unseen_indexes = set(range(len(filtration))) - seen_indexes
    for i in unseen_indexes:
        barcode = (filtration[i].dim, i, -1)  # (index, death index = ∞, dimension)
        barcodes.append(barcode)

    barcodes.sort(key=lambda x: (x[0], x[1], x[2] if x[2] != -1 else float('inf')))  # Sort by (dimension, birth index)
    return barcodes


def print_barcodes(barcodes : list[tuple[int, int, int]], filtration : list[Simplex]) -> None:
    for (dim, birth_idx, death_idx) in barcodes:
        birth_time = filtration[birth_idx].val
        death_time = filtration[death_idx].val if death_idx != -1 else float('inf')
        print(f"{dim} {birth_time} {death_time}")

print_barcodes(extract_barcodes(reduce_boundary_matrix(boundary_matrix(filtration)), filtration), filtration)
print(extract_barcodes(reduce_boundary_matrix(boundary_matrix(filtration)), filtration))

Seen indexes: {1, 2, 3, 4, 5, 6, 8, 9}
0 1.0 inf
0 1.0 2.0
0 1.0 2.0
0 3.0 4.0
1 4.0 inf
1 5.0 6.0
Seen indexes: {1, 2, 3, 4, 5, 6, 8, 9}
[(0, 0, -1), (0, 1, 3), (0, 2, 4), (0, 5, 6), (1, 7, -1), (1, 8, 9)]


In [7]:

## TODO : report, answer questions, complexity analysis, plots, analysis of graphs, 2 3 pages. 
# >>> jupyter notebook

In [ ]:
## Question 5 - Complexity analysis
filtration_a = read_filtration("filtrations/filtration_a.txt")
print("Filtration a:")
print_barcodes(extract_barcodes(reduce_boundary_matrix(boundary_matrix(filtration_a))), filtration_a)